In [2]:
from transformers import pipeline
import pandas as pd
import utilsNb as utils
import numpy as np
from tqdm import tqdm

In [3]:
ESG_DICT = {
    "environement": ["changement climatique", "capital naturel", "pollution et déchets", "Opportunités environnementales"],
    "social": ["capital humain", "responsabilité du produit", "opposition des parties prenantes", "opportunités sociales", "services publiques"],
    "gouvernance": ["gouvernance d'entreprises", "comportement d'entreprises"],
    "non-esg": ["autre","divertissement"]
}

CANDIDATE_LABELS = list(ESG_DICT.values())
CANDIDATE_LABELS = [item for sublist in CANDIDATE_LABELS for item in sublist]


classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")


In [4]:
def predict_sample(sample, candidate_labels, top_k = 3, return_all = False):
    output = classifier(sample, candidate_labels, multi_label=False)
    output_df = pd.DataFrame(output)[["labels", "scores"]][:top_k]
    # find the key from the esg_dict
    output_df["key_label"] = output_df["labels"].apply(lambda x: [key for key, value in ESG_DICT.items() if x in value][0])
    
    count_dict = {}
    for label in output_df["key_label"]:
        count_dict[label] = count_dict.get(label, 0) + 1

    best_choice = max(count_dict, key=count_dict.get)
    
    if return_all: return output_df, best_choice
    
    return best_choice



In [5]:
smp = "Après études, il a été décidé de ne \npas  le  réaliser,  car  ce  genre  de  parking  est  un  aspirateur  à  voitures. Il  n’est  pas  contre  les \nvoitures,  mais  il  trouve  qu’à  cet  endroit,  il  est  inutile  de  faire  cette  démarche. Raison  pour \nlaquelle il invite le Conseil à ne pas voter cet amendement. \n\nMme Brigitte PEREZ demande le vote nominal. Plus d’un cinquième des membres soutiennent \ncette demande. \n\n9 M."

predict_sample(smp, CANDIDATE_LABELS, return_all=True, top_k=10)

(                             labels    scores     key_label
 0         responsabilité du produit  0.309370        social
 1  opposition des parties prenantes  0.237279        social
 2                             autre  0.173625       non-esg
 3                    capital humain  0.079359        social
 4        comportement d'entreprises  0.069967   gouvernance
 5                services publiques  0.026884        social
 6         gouvernance d'entreprises  0.025327   gouvernance
 7              pollution et déchets  0.017591  environement
 8                    divertissement  0.017427       non-esg
 9                   capital naturel  0.015270  environement,
 'social')

In [6]:
df = pd.read_csv("./pv_2312/pv2312_512_predictions.csv",header = 0)
df.head()

,section_number,text,cbl_pred,cbl_1024_pred
0,1,1. \n\nAppel : \n\n93 Conseillères et Conseil...,non-esg,gouvernance
1,2,2. \n\nProcès-verbal de la séance du 13 novemb...,non-esg,non-esg
2,3,3. \n\nApprobation de l’ordre du jour \n\nM. l...,non-esg,non-esg
3,4,4. \n\nCommunications du Bureau \n\n• M. le ...,non-esg,gouvernance
4,4,"Or, il a appris ce matin qu’un membre du Conse...",non-esg,gouvernance


In [ ]:
df['0-shot'] = None
for i,row in tqdm(df.iterrows(), total = len(df)):
    df.at[i,'0-shot'] = predict_sample(row['text'], CANDIDATE_LABELS)   

In [14]:
# df.to_csv("./pv_2312/pv512_predictions_0shot.csv", index = False)

# Zero shot classification with machine translation

In [23]:
# Use a pipeline as a high-level helper
from transformers import pipeline
from easynmt import EasyNMT
import warnings
warnings.filterwarnings('ignore')

pipe = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-large-zeroshot-v1.1-all-33")
model = EasyNMT('opus-mt',cache_folder="./NMT_model_cache")


In [ ]:
df['0-shot-EN'] = None
for i,row in tqdm(df.iterrows(), total = len(df)):
    fr_text = row['text']
    en_text = model.translate(fr_text, source_lang='fr', target_lang='en')
    df.at[i,'0-shot-EN'] = predict_sample(en_text, CANDIDATE_LABELS)


In [ ]:
df.to_csv("./pv_2312/pv512_predictions_0shot_x2.csv", index = False)